# Import

In [1]:
import pandas as pd

# Load Data

In [2]:
df = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_1999_2023.pkl")

# Prep Data

In [3]:
df_wr = df.copy().query("position == 'WR'")

In [4]:
df_wr.dropna(subset="Draft Year", inplace=True)

In [5]:
df_wr['Draft Year'] = df_wr['Draft Year'].astype(str).str.replace('s', '')
df_wr["Draft Year"] = df_wr["Draft Year"].astype(int)

In [6]:
df_wr.head()

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,avg_cushion,avg_separation,avg_intended_air_yards_receiving,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation
22,00-0000172,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,00-0000387,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,00-0000387,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,00-0000387,2001,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,00-0000422,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create Rook Year DF

In [7]:
df_rook_year = df_wr.copy().loc[df_wr["season"] == df_wr["Draft Year"]]

In [10]:
len(df_rook_year)

775

# Mode Career Finish by Target Bucket

In [11]:
# Calculate the mode of the 'tier' for each player
mode_tiers = df_wr.groupby('player_id')['tier'].agg(lambda x: pd.Series.mode(x).iloc[0]).reset_index()
mode_tiers.rename(columns={'tier': 'mode_tier'}, inplace=True)

# Merge the mode tiers back into the rookie year DataFrame
df_rook_year_update = df_wr[df_wr["season"] == df_wr["Draft Year"]].merge(mode_tiers, on='player_id', how='left')

# Define target categories with accurate labels
bins = [0, 20, 40, 60, 80, 100, 120, 140, float('inf')]
labels = ['0-19', '20-39', '40-59', '60-79', '80-99', '100-119', '120-139', '140+']

# Categorize the targets
df_rook_year_update['target_categories'] = pd.cut(df_rook_year_update['targets'], bins=bins, labels=labels, right=False)

# Group by 'target_categories' and 'mode_tier', then count the occurrences
result = df_rook_year_update.groupby(['target_categories', 'mode_tier']).size().unstack(fill_value=0)

# Display the result
result

/tmp/ipykernel_53443/1948408812.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result = df_rook_year_update.groupby(['target_categories', 'mode_tier']).size().unstack(fill_value=0)


mode_tier,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
target_categories,,,,,,,,,,,,,,,,,,,
0-19,6,6,2,6,6,14,12,19,22,21,32,36,36,46,44,25,33,12,3
20-39,7,1,8,6,9,9,10,14,13,15,16,13,12,4,2,2,2,1,0
40-59,6,5,11,7,8,5,11,12,10,5,0,1,1,0,0,0,1,0,0
60-79,6,10,8,9,16,9,7,3,0,2,1,0,0,1,0,0,0,0,0
80-99,9,11,7,11,3,6,2,2,1,0,1,0,0,0,0,0,0,0,0
100-119,8,6,4,4,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
120-139,5,2,1,3,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
140+,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
row_totals = result.sum(axis=1)
result_percentage_by_row = round(result.div(row_totals, axis=0) * 100, 2)
result_percentage_by_row

mode_tier,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
target_categories,,,,,,,,,,,,,,,,,,,
0-19,1.57,1.57,0.52,1.57,1.57,3.67,3.15,4.99,5.77,5.51,8.40,9.45,9.45,12.07,11.55,6.56,8.66,3.15,0.79
20-39,4.86,0.69,5.56,4.17,6.25,6.25,6.94,9.72,9.03,10.42,11.11,9.03,8.33,2.78,1.39,1.39,1.39,0.69,0.00
40-59,7.23,6.02,13.25,8.43,9.64,6.02,13.25,14.46,12.05,6.02,0.00,1.20,1.20,0.00,0.00,0.00,1.20,0.00,0.00
60-79,8.33,13.89,11.11,12.50,22.22,12.50,9.72,4.17,0.00,2.78,1.39,0.00,0.00,1.39,0.00,0.00,0.00,0.00,0.00
80-99,16.98,20.75,13.21,20.75,5.66,11.32,3.77,3.77,1.89,0.00,1.89,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
100-119,33.33,25.00,16.67,16.67,0.00,0.00,4.17,0.00,4.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
120-139,38.46,15.38,7.69,23.08,0.00,0.00,7.69,7.69,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
140+,40.00,60.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [16]:
df_rook_year.query("targets >= 140")[["player_name", "targets", "receiving_yards"]]

,player_name,targets,receiving_yards
4229,Anquan Boldin,165,1377.0
9821,Kelvin Benjamin,145,1008.0
12809,Jaylen Waddle,140,1015.0
13093,Garrett Wilson,147,1103.0
13241,Puka Nacua,160,1486.0


# The Great Filter

In [13]:
# Calculate the mode of the 'tier' for each player
mode_tiers = df_wr.groupby('player_id')['tier'].agg(lambda x: pd.Series.mode(x).iloc[0]).reset_index()
mode_tiers.rename(columns={'tier': 'mode_tier'}, inplace=True)

# Merge the mode tiers back into the rookie year DataFrame
df_rook_year_update = df_wr[df_wr["season"] == df_wr["Draft Year"]].merge(mode_tiers, on='player_id', how='left')

# Define target categories with accurate labels
bins = [0, 75, float('inf')]
labels = ['0-74', '75+']

# Categorize the targets
df_rook_year_update['target_categories'] = pd.cut(df_rook_year_update['targets'], bins=bins, labels=labels, right=False)

# Group by 'target_categories' and 'mode_tier', then count the occurrences
result = df_rook_year_update.groupby(['target_categories', 'mode_tier']).size().unstack(fill_value=0)

# Display the result
result

/tmp/ipykernel_53443/2009623244.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result = df_rook_year_update.groupby(['target_categories', 'mode_tier']).size().unstack(fill_value=0)


mode_tier,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
target_categories,,,,,,,,,,,,,,,,,,,
0-74,25,18,26,25,35,35,38,48,45,43,49,50,49,51,46,27,36,13,3
75+,24,26,15,21,7,8,6,3,2,0,1,0,0,0,0,0,0,0,0


In [14]:
row_totals = result.sum(axis=1)
result_percentage_by_row = round(result.div(row_totals, axis=0) * 100, 2)
result_percentage_by_row

mode_tier,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
target_categories,,,,,,,,,,,,,,,,,,,
0-74,3.78,2.72,3.93,3.78,5.29,5.29,5.74,7.25,6.80,6.5,7.40,7.55,7.4,7.7,6.95,4.08,5.44,1.96,0.45
75+,21.24,23.01,13.27,18.58,6.19,7.08,5.31,2.65,1.77,0.0,0.88,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00


In [15]:
df_rook_year[["targets", "receiving_yards"]].corr()

,targets,receiving_yards
targets,1.000000,0.948709
receiving_yards,0.948709,1.000000


# Filters on Rook Year

In [ ]:
df_rook_year.query("targets >= 100")[["Draft Year", "player_name", "targets"]]

In [ ]:
def highlight_row(row):
    return ['background-color: red' if row['player_name'] == 'Michael Clayton' else '' for _ in row]

# Assuming 'df' is your DataFrame
df_highlighted = df_filter.style.apply(highlight_row, axis=1)
df_highlighted